In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [33]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromFile2") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

In [42]:
lines = spark.readStream \
.format("socket") \
.option("host","172.17.0.3") \
.option("port", 9999) \
.load()

In [43]:
words = lines.select(f.explode(f.split(f.col("value"), " ")).alias("word"))

In [44]:
word_counts = words.groupBy("word").count().sort(f.desc("count"))

Open Port
docker run -i --rm --network=bridge  subfuzion/netcat -vl 9999

In [45]:
query = word_counts.writeStream \
.outputMode("complete") \
.format("console") \
.start()

In [ ]:
query.awaitTermination()

In [40]:
query.stop()